In [25]:
import pandas as pd
import numpy as np
import re
import emoji

# Step 1: Load the scraped CSV (from data_scraping folder)
file_path = "../data_scraping/scraped_airlinequality_ethiopian_airlines_reviews.csv"
df = pd.read_csv(file_path)

# Step 2: Drop unwanted columns
columns_to_drop = ['type_of_traveller', 'seat_type', 'route', 'date_flown', 'recommended', 'wifi_and_connectivity', 'aircraft']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# Step 3: Fix 'Date Flown' format ➝ 'Mon-YYYY'
def fix_date(val):
    try:
        parsed = pd.to_datetime(val, format='%b-%y')
        return parsed.strftime('%b-%Y')
    except:
        return val

if 'Date Flown' in df.columns:
    df['Date Flown'] = df['Date Flown'].astype(str).str.strip().apply(fix_date)

# Step 4: Add 'Customer service' column (avg of cabin_staff_service + ground_service)
if 'cabin_staff_service' in df.columns or 'ground_service' in df.columns:
    df['Customer service'] = df[['cabin_staff_service', 'ground_service']].apply(
        lambda row: (
            np.mean([val for val in [row.get('cabin_staff_service'), row.get('ground_service')] if pd.notnull(val)])
            if pd.notnull(row.get('cabin_staff_service')) or pd.notnull(row.get('ground_service')) else np.nan
        ),
        axis=1
    )

# Step 5: Drop now-unneeded columns
df.drop(columns=[col for col in ['Unnamed: 11', 'cabin_staff_service', 'ground_service'] if col in df.columns], inplace=True)

# Step 6: Round 'Customer service'
df['Customer service'] = df['Customer service'].apply(lambda x: round(x) if pd.notnull(x) else np.nan)

# Step 7: Fill missing values in rating columns
rating_columns = ['inflight_entertainment', 'food_and_beverages', 'seat_comfort', 'Customer service']
for col in rating_columns:
    if col in df.columns and df[col].isnull().any():
        median_value = round(df[col].median())
        df[col] = df[col].fillna(median_value)

# Step 8: Clean 'comment' column
df['comment'] = df['comment'].astype(str).str.split('|').str[-1].str.strip()

# Step 9: Clean 'title' column
df['title'] = df['title'].astype(str).replace({
    'â€œ': '', 'â€': '', '“': '', '”': '', '"': ''
}, regex=True).str.strip()


# Step 10: Extract 'Month' and 'Year' from 'Date Flown'
df['Date Flown'] = df['Date Flown'].fillna('unknown-unknown')
df[['Month', 'Year']] = df['Date Flown'].str.split('-', expand=True)
df['Year'] = df['Year'].apply(lambda x: '20' + x if pd.notnull(x) and str(x).isdigit() and len(str(x)) == 2 else x)
df.drop(columns='Date Flown', inplace=True)
df.insert(0, 'Year', df.pop('Year'))
df.insert(0, 'Month', df.pop('Month'))

# Step 11: Extract 'From' and 'Destination' from 'Route'
df['Route'] = df['Route'].fillna('unknown to unknown')
df[['From', 'Destination']] = df['Route'].str.split(' to ', expand=True)
df.drop(columns='Route', inplace=True)
df.insert(df.columns.get_loc('comment'), 'Destination', df.pop('Destination'))
df.insert(df.columns.get_loc('Destination'), 'From', df.pop('From'))

# Step 12: Clean 'Destination' column
df['Destination'] = df['Destination'].astype(str).str.split(' via').str[0].str.strip()

# Step 13: Drop extra columns
df.drop(columns=[col for col in ['author_country', 'Unnamed: 12'] if col in df.columns], inplace=True)

# Step 14: Normalize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.rename(columns={
    'title': 'review_title',
    'from': 'departure_city',
    'destination': 'arrival_city',
    'comment': 'review_comment',
    'customer_service': 'customer_service',
}, inplace=True)


# Step 15: Remove rows with 'unknown' month
df = df[df['month'].astype(str).str.strip().str.lower() != 'unknown'].reset_index(drop=True)


# Step 15.1: Drop duplicate rows
df.drop_duplicates(inplace=True)


# Step 15.2: Clean review text fields
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = str(text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = emoji.replace_emoji(text, replace='')
    text = text.replace('-', ' ').replace('&', 'and')
    text = re.sub(r'[-–—]', ' ', text)
    text = re.sub(r'[^\w\s/]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^A-Za-z0-9\s\.\,\?\!\/]', '', text)
    return text

columns_to_clean = ['review_title', 'departure_city', 'arrival_city', 'review_comment']
for col in columns_to_clean:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)



# Step 16: Add source column
df['source'] = 'AirlineQuality'

# Step 17: Save cleaned result to current folder
output_path = "cleaned_airlinequality_ethiopian_airlines.csv"

df.to_csv(output_path, index=False)

print("✅ Cleaned data saved to:", output_path)


✅ Cleaned data saved to: C:\Users\abro27\OneDrive\Desktop\Mak\Education\3.Data_Analytics\Final Project\Capstone Projects\Datas\Airlinequality\cleaned_airlinequality_ethiopian_airlines.csv
